In [1]:
import pandas as pd
from tqdm import tqdm
from pprint import pprint

pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)
from crossref_commons.retrieval import get_publication_as_json
from crossref_commons.iteration import iterate_publications_as_json

from utils import remove_html_tags, split_entries_in_column, extract_FKZ

/Users/isaacduong/BMBF_Clustering_Project/BMBFenv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
BMBF = pd.read_csv("../data/cleanedBMBF.csv")
BMBF.head(2)

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,Staat,Ausführende Stelle,Gemeindekennziffer.1,Stadt/Gemeinde.1,Ort.1,Bundesland.1,Staat.1,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,Universität Rostock - Fachbereich Elektrotechnik und Informationstechnik,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,"Verbundprojekt MESSIN: Entwicklung und Systemintegration der Komponenten Navigation, Automatische Steuerung, Kommunikation und Energieversorgung für den Meßdelphin MESSIN",FB4010,"Technik-/Infrastrukturentwicklung und -bereitstellung (Überwachungssysteme, Begleitforschung zum Monitoring, innovative Technik und Geräteträger)",01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR
1,D251800/0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,KACO new energy GmbH,8121000.0,Heilbronn,Heilbronn,Baden-Württemberg,Deutschland,Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR


### At CROSSREF, each funder is assigned to a so-called funder number, which is unique and is used to identify the oganisation/ funder . So we will retrieve all metadatas which are related to the funder of each value in column "Ressort" as below. To do this, we make use of the python modul crossref-commons (https://api.crossref.org/swagger-ui/index.html)

## BMBF : 501100002347 Bundesministerium für Bildung und Forschung 
## BMWi : 501100002765 Bundesministerium für Wirtschaft und Technologie
## BMU : 501100013549 Bundesministerium für Umwelt, Naturschutz, nukleare Sicherheit und Verbraucherschutz
## BMVI : 100008383  Bundesministerium für Verkehr und Digitale Infrastruktur
## BMEL : 501100020631 Bundesministerium Ernährung und Landwirtschaft
BMJV_BLE : no funder number found

In [32]:
# create a python dictionary of the funders
funders = {"BMBF":"501100002347", "BMWi":"501100002765", "BMU":"501100013549", "BMVI":"100008383", "BMEL":"501100020631"}

### make a test request and print out to examine the response structure

In [5]:
for i,p in enumerate(iterate_publications_as_json(max_results=1, filter={'funder':"501100002347"})):
    pprint(p)

{'DOI': '10.1109/cist49399.2021.9357189',
 'URL': 'http://dx.doi.org/10.1109/cist49399.2021.9357189',
 'author': [{'affiliation': [],
             'family': 'Torge',
             'given': 'Sunna',
             'sequence': 'first'},
            {'affiliation': [],
             'family': 'Hahn',
             'given': 'Waldemar',
             'sequence': 'additional'},
            {'affiliation': [],
             'family': 'Jakel',
             'given': 'Rene',
             'sequence': 'additional'},
            {'affiliation': [],
             'family': 'Nagel',
             'given': 'Wolfgang E.',
             'sequence': 'additional'}],
 'container-title': ['2020 6th IEEE Congress on Information Science and '
                     'Technology (CiSt)'],
 'content-domain': {'crossmark-restriction': False, 'domain': []},
 'created': {'date-parts': [[2021, 3, 29]],
             'date-time': '2021-03-29T21:32:53Z',
             'timestamp': 1617053573000},
 'deposited': {'date-parts': [[2021

In [6]:
def retrieve_metadata(funder, max_results = 50200):
    """ function to retrieve metadatas from publication funded by BMBF.
        BMBF has most search results 50135 on 15.11.2023, 
        that´s why max_results = 50200. Other funder have much less serch results
    """
    
    metadatas = []
    filters = {'funder': funder}
    for i, p in tqdm(enumerate(iterate_publications_as_json(max_results=max_results, filter=filters))):
        pub = {}
        for f in p.get('funder'):
            if f.get('DOI') == f'10.13039/{funder}':
                pub['AWARD'] = f.get('award','')
                break
        pub["FUNDER"] = funder
        pub['DOI'] = p.get('DOI','')
        pub['TITLE'] = p.get('title','')
        pub['ABSTRACT'] = p.get('abstract','')
        pub['LANG'] = p.get('language','')
        
        if ('resource' in p.keys()) and ('primary' in p['resource'].keys()):
            pub['RESOURCE'] = p['resource']['primary'].get('URL','')
        else:
            pub['RESOURCE']=''

        metadatas.append(pub)
 
    return metadatas


In [8]:
def create_metadatas_table(funders):
    
    total_metadatas = []
    for funder, funder_number in funders.items():
        datas = retrieve_metadata(funder=funder_number)
        total_metadatas.extend(datas)

    return pd.DataFrame.from_records(total_metadatas)

In [33]:
metadatas = create_metadatas_table(funders)

419it [00:09, 45.89it/s]
122it [00:05, 20.76it/s]


In [9]:
# save data to csv file before processing the data
metadatas.to_csv("../data/crossref_pub_metadatas.csv",index = False)

In [10]:
# 51953 publications in total 
metadatas.shape

(51953, 7)

In [11]:
# but only 325 publications in German language
metadatas[metadatas["LANG"]=="de"].shape

(325, 7)

In [50]:
# create a table that only contains publications in German language
de_metadatas = metadatas[metadatas["LANG"]=="de"]

In [39]:
# only 253 records have values in column "AWARD"
# which represent the FKZs we need to match the enrichment 
de_metadatas[de_metadatas["AWARD"] !=''].shape

(253, 7)

In [59]:
# 127 records do not have an abstract
de_metadatas[de_metadatas["ABSTRACT"] ==""].shape

(127, 7)

In [60]:
de_metadatas = de_metadatas[de_metadatas["AWARD"] !='']


In [62]:
# perform some data cleaning 
de_metadatas["ABSTRACT"] = de_metadatas["ABSTRACT"].apply(remove_html_tags)
de_metadatas["AWARD"] = de_metadatas["AWARD"].apply(lambda li: li[0]) 
de_metadatas = split_entries_in_column(de_metadatas, "AWARD")
de_metadatas

,AWARD,FUNDER,DOI,TITLE,ABSTRACT,LANG,RESOURCE
77,[,501100002347,10.1007/s11618-023-01144-0,"['Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen\xa0– ein systematischer Literaturüberblick', 'Overall conditions and challenges of arts and cultural education in rural areas—a\xa0systematic review']","ZusammenfassungKulturelle Bildung ist auf eine vielfältige Angebotsstruktur und die Kooperationen unterschiedlicher Bildungs- und Kultureinrichtungen angewiesen, um kulturelle Teilhabe zu ermöglichen. Die intensive Förderung durch öffentliche, zivilgesellschaftliche und private Akteur*innen hat in den letzten Jahren zu einem Ausbau von formalen und non-formalen Bildungsangeboten geführt. Gleichzeitig finden sich zunehmend Forschungsarbeiten auf dem Gebiet, deren Fokus weiterhin stärker auf städtische Bildungskontexte als auf die Bedingungen ländlicher Räume gerichtet ist. Vor allem fehlt bislang eine systematische Bestandsaufnahme der vorhandenen Studien und Erkenntnisse zu kultureller Bildung speziell in ländlichen Räumen. Der vorliegende systematische Review setzt hier an und erarbeitet einen umfassenden Überblick über Publikationen zu kultureller Bildung, Kulturarbeit und Soziokultur in ländlichen Räumen. Inhaltlich steht bei dieser Untersuchung die Aufarbeitung der Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen im Mittelpunkt. Die Beschreibung zentraler Themen erfolgt dabei entlang von drei Teilbereichen: kommunal- und kulturpolitischen Bedingungen, (infra-) strukturellen Faktoren sowie kulturellen und gesellschaftlichen Dimensionen.",de,https://link.springer.com/10.1007/s11618-023-01144-0
149,[,501100002347,10.1007/s11618-021-00993-x,['Ein Instrument zur Messung von Lerntransferbedingungen im Kontext der hochschuldidaktischen Weiterbildung'],"ZusammenfassungDie hochschuldidaktische Qualifizierung von Lehrenden hat insbesondere mit dem Bund-Länder-Programm Qualitätspakt Lehre und den in diesem Kontext entstandenen Angeboten an Bedeutung gewonnen. Zum Transfer des in den Qualifizierungen Gelernten in den Lehralltag und seinen Bedingungen liegen jedoch nur wenige Befunde vor. Der vorliegende Aufsatz stellt ein im Anschluss an den Diskurs der Lerntransferforschung entwickeltes und auf den deutschsprachigen, hochschuldidaktischen Bereich zugeschnittenes Instrument zur Erfassung von Lerntransferbedingungen vor, das die Dimensionen Transfermotivation, Selbstwirksamkeitserwartung, persönliche Transferkapazitäten, Trainingsdesign, Lehrbedingungen, Möglichkeit zur Anwendung, positive Folgen, Unterstützung durch Kollegen, Unterstützung durch Vorgesetzten, Transferklima und Unterstützung durch Studierende erfasst. Die auf konfirmatorischen Faktorenanalysen basierenden Auswertungen geben Hinweise auf die Konstruktvalidität des Instruments, das sowohl zur Evaluation des Lerntransfers und seinen Bedingungen im Kontext hochschuldidaktischer Weiterbildungen als auch zur Weiterentwicklung des diesbezüglichen Forschungsstandes geeignet ist.",de,https://link.springer.com/10.1007/s11618-021-00993-x
174,[,501100002347,10.1002/cite.201900046,['Funktionalisierung von Ultrafiltrationsmembranen zur Integration von spezifischen Adsorbereigenschaften'],AbstractDas Anwendungspotenzial der Ultrafiltration (UF) zur Wasserreinigung kann durch die Einbindung eines Adsorberpolymers für die Komplexierung von Schwermetallen in die Porenstruktur einer UF‐Membran erweitert werden. Der Lösungsweg in dieser Arbeit basiert auf der Funktionalisierung eines Membran‐ und eines Adsorberpolymers mit komplementären Reaktivgruppen und der kovalenten Immobilisierung des Adsorberpolymers in der porösen Stützschicht der UF‐Membran durch Click‐Reaktion. Die Trennleistungen und Membrancharakteristika der hergestellten Membranen sind mit denen konventioneller UF‐Membranen vergleichbar.,de,https://onlinelibrary.wiley.com/doi/10.1002/cite.201900046
354,[,501100002347,10.1055/a-1082-0740,['Individual-Level Linkage of Prim

In [63]:
de_metadatas["AWARD"].shape

(253,)

In [64]:
# extract FKZ from word such es Förderkennzeichen
de_metadatas["AWARD"] = de_metadatas["AWARD"].apply(extract_FKZ)


In [65]:
# enpack list column TITLE  
de_metadatas["TITLE"] = de_metadatas["TITLE"].apply( lambda li: li[0])


In [66]:
# join both columns TITLE and ABSTRACT 
de_metadatas.loc[:,"crossref_enrichment"] = de_metadatas.loc[:,"TITLE"] + " " + de_metadatas.loc[:,"ABSTRACT"] 
de_metadatas

,AWARD,FUNDER,DOI,TITLE,ABSTRACT,LANG,RESOURCE,crossref_enrichment
77,None,501100002347,10.1007/s11618-023-01144-0,[,"ZusammenfassungKulturelle Bildung ist auf eine vielfältige Angebotsstruktur und die Kooperationen unterschiedlicher Bildungs- und Kultureinrichtungen angewiesen, um kulturelle Teilhabe zu ermöglichen. Die intensive Förderung durch öffentliche, zivilgesellschaftliche und private Akteur*innen hat in den letzten Jahren zu einem Ausbau von formalen und non-formalen Bildungsangeboten geführt. Gleichzeitig finden sich zunehmend Forschungsarbeiten auf dem Gebiet, deren Fokus weiterhin stärker auf städtische Bildungskontexte als auf die Bedingungen ländlicher Räume gerichtet ist. Vor allem fehlt bislang eine systematische Bestandsaufnahme der vorhandenen Studien und Erkenntnisse zu kultureller Bildung speziell in ländlichen Räumen. Der vorliegende systematische Review setzt hier an und erarbeitet einen umfassenden Überblick über Publikationen zu kultureller Bildung, Kulturarbeit und Soziokultur in ländlichen Räumen. Inhaltlich steht bei dieser Untersuchung die Aufarbeitung der Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen im Mittelpunkt. Die Beschreibung zentraler Themen erfolgt dabei entlang von drei Teilbereichen: kommunal- und kulturpolitischen Bedingungen, (infra-) strukturellen Faktoren sowie kulturellen und gesellschaftlichen Dimensionen.",de,https://link.springer.com/10.1007/s11618-023-01144-0,"[ ZusammenfassungKulturelle Bildung ist auf eine vielfältige Angebotsstruktur und die Kooperationen unterschiedlicher Bildungs- und Kultureinrichtungen angewiesen, um kulturelle Teilhabe zu ermöglichen. Die intensive Förderung durch öffentliche, zivilgesellschaftliche und private Akteur*innen hat in den letzten Jahren zu einem Ausbau von formalen und non-formalen Bildungsangeboten geführt. Gleichzeitig finden sich zunehmend Forschungsarbeiten auf dem Gebiet, deren Fokus weiterhin stärker auf städtische Bildungskontexte als auf die Bedingungen ländlicher Räume gerichtet ist. Vor allem fehlt bislang eine systematische Bestandsaufnahme der vorhandenen Studien und Erkenntnisse zu kultureller Bildung speziell in ländlichen Räumen. Der vorliegende systematische Review setzt hier an und erarbeitet einen umfassenden Überblick über Publikationen zu kultureller Bildung, Kulturarbeit und Soziokultur in ländlichen Räumen. Inhaltlich steht bei dieser Untersuchung die Aufarbeitung der Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen im Mittelpunkt. Die Beschreibung zentraler Themen erfolgt dabei entlang von drei Teilbereichen: kommunal- und kulturpolitischen Bedingungen, (infra-) strukturellen Faktoren sowie kulturellen und gesellschaftlichen Dimensionen."
149,None,501100002347,10.1007/s11618-021-00993-x,[,"ZusammenfassungDie hochschuldidaktische Qualifizierung von Lehrenden hat insbesondere mit dem Bund-Länder-Programm Qualitätspakt Lehre und den in diesem Kontext entstandenen Angeboten an Bedeutung gewonnen. Zum Transfer des in den Qualifizierungen Gelernten in den Lehralltag und seinen Bedingungen liegen jedoch nur wenige Befunde vor. Der vorliegende Aufsatz stellt ein im Anschluss an den Diskurs der Lerntransferforschung entwickeltes und auf den deutschsprachigen, hochschuldidaktischen Bereich zugeschnittenes Instrument zur Erfassung von Lerntransferbedingungen vor, das die Dimensionen Transfermotivation, Selbstwirksamkeitserwartung, persönliche Transferkapazitäten, Trainingsdesign, Lehrbedingungen, Möglichkeit zur Anwendung, positive Folgen, Unterstützung durch Kollegen, Unterstützung durch Vorgesetzten, Transferklima und Unterstützung durch Studierende erfasst. Die auf konfirmatorischen Faktorenanalysen basierenden Auswertungen geben Hinweise auf die Konstruktvalidität des Instruments, das sowohl zur Evaluation des Lerntransfers und seinen Bedingungen im Kontext hochschuldidaktischer Weiterbildungen als auch zur Weiterentwicklung des diesbezüglichen Forschungsstan

In [ ]:
# duplicated in "AWARD" columns indicates that there are more than 1 publication/ press release for some topics
de_metadatas.AWARD.value_counts()

In [68]:
de_metadatas.shape

(253, 8)

In [ ]:
# create a dataframe, which merge all publications / press releases with same funder number.
crossref_enrichment = de_metadatas.groupby('AWARD')['crossref_enrichment'].agg(lambda x: ' '.join(x)).reset_index()

In [ ]:
crossref_enrichment.to_csv("../data/BMBF_crossref_enrichment.csv",index=False)


In [4]:
crossref_abstract["LANG"].value_counts(dropna=False)

LANG
en     45465
NaN     6157
de       325
es         3
fr         1
ru         1
pl         1
Name: count, dtype: int64